## Features:
- `LEN`: number of words in tuple (ranges from 1 to 5)
- `HAS_NN`: boolean indicating if the 
- `DEP_PATT`: string dependency pattern
- `PREV_2_W`: previous 2 words
- `NEXT_2_W`: next 2 words
- `NEXT_W`: next word
- `PREV_W`: next word
- `PREV_2_T`: previous 2 tags
- `NEXT_2_T`: next 2 tags
- `NEXT_T`: next tag
- `PREV_T`: prev tag
- `DOC_FR`: document frequency
- `IS_TITLE`: boolean indicating is this tuple from the title(true) or the post(false)
- `REL_POS`: proportion of sentence preceding the first element of the tuple